# Imports

In [8]:
import numpy as np 
import matplotlib.pyplot as plt  
import pandas as pd
import scipy.stats as stats 
import sympy as sy 
import time

# Constants

In [ ]:
# We want to take time horizon as 100 years which is 36525 days including the years with more or less days
TIME_HORIZON = 5 # TODO: this time horizon should be equal to 36525
LAMBDA = 4
CLAIMSIZE_MEAN = 16000 

### Use Monte Carlo simulation to investigate the influence of the premium c and the initial capital u on the ruin probability ψ(u).

In [ ]:
# 1 - Create the distributions for the claim sizes, number of claims

# 1.1 - Claim sizes (Uniform distribution with a mean of 16,000 and a variance of 12,000,000)
# 1.1.1 - Calculate the a and b of the continuous uniform distribution
a, b, mu, var = sy.symbols("a, b, mu, sigma^2")

eq1 = sy.Eq((a + b) / 2, mu)
eq2 = sy.Eq(((a - b) ** 2) / 12, var)

# by using the two equations which are the definitions of mean and variance of continuous uniform distribution,
# we find the values of a and b
bValue = sy.solve(
    eq1.subs({a: sy.solve(eq2, a)[0], mu: CLAIMSIZE_MEAN, var: 12000000})
)[0]
aValue = sy.solve(eq1.subs({b: bValue, mu: CLAIMSIZE_MEAN}), a)[0]
print(f"Claim Sizes ~ U({aValue},{bValue})")

# 1.1.2 Create the claim size distribution
claimSizeDistribution = stats.uniform(loc=aValue, scale=bValue - aValue)

# 1.2 - Number of Claims (Poisson(lambda * t)) (lambda is 4 per day)
# NOTE: In the example, the expected value of N(t) was taken by saying time horizon * lambda however,
# we decided to sample the number of claims as well since it is an RV.
NumberOfClaimDistribution = stats.poisson(mu=TIME_HORIZON * LAMBDA)

Claim Sizes ~ U(10000,22000)


In [ ]:
class question_1:
    def run(self, u0, c):
        # we first get a sample from the number of claims RV N(t) ~ Pois(lam * t)
        N = NumberOfClaimDistribution.rvs(1)
        # We get interarrival times ~ Expon(lambda) which is a property of PP
        interArrivals = stats.expon(scale=1 / LAMBDA).rvs(N)
        arrivalTimes = np.cumsum(interArrivals)

        # We sample the claim sizes
        claimSizes = claimSizeDistribution.rvs(N)
        cumulativeClaims = np.cumsum(claimSizes)

        # Calculate U(t) at every t
        levels = u0 + arrivalTimes * c - cumulativeClaims

        ruin = min(levels) < 0
        return arrivalTimes, cumulativeClaims, levels, ruin

    def simulate(self, u0List, thetaList, n=1000):

        print(
            f"""
Time Horizon: {TIME_HORIZON} days.
u0 List: {u0List}
Safety Loading List: {thetaList}

"""
        )

        result = [[[]] * len(thetaList)] * len(u0List)

        for i, u0 in enumerate(u0List):
            for j, theta in enumerate(thetaList):
                result[i][j] = self.simulate_one_pair(u0, theta, n)

        df = pd.DataFrame(data=result, columns=thetaList, index=u0List)
        df.to_csv('question_1_simulation_results.csv', index=False)
        
        return df
    def simulate_one_pair(self, u0, theta, n):
        start = time.time()
        result = np.zeros(n)
        c = (1 + theta) * LAMBDA * CLAIMSIZE_MEAN  # CLAIMSIZE_MEAN is E(X_i)

        for i in range(n):
            arrivalTimes, cumulativeClaims, levels, ruin = self.run(u0, c)
            result[i] = 1 if ruin == True else 0

        end = time.time()
        print(
            f"Time taken for {n} simulations: {end - start} seconds, this is equal to {(end - start) / n} seconds per run."
        )
        return np.mean(result)

    def simulate_once_and_plot(self, u0, theta):
        c = (1 + theta) * LAMBDA * CLAIMSIZE_MEAN  # CLAIMSIZE_MEAN is E(X_i)
        arrivalTimes, cumulativeClaims, levels, ruin = self.run(u0, c)

        plt.figure()
        plt.step(arrivalTimes, cumulativeClaims, "b", where="post")
        plt.plot(arrivalTimes, u0 + c * arrivalTimes, "r")
        plt.show()

In [ ]:
# TODO: Uncomment this. 
# u0List = [CLAIMSIZE_MEAN * i for i in range(10)]
# thetaList = [0, 0.1, 0.5, 0.9, 1]
# question_1().simulate(u0List, thetaList)

### Determine the distribution of the maximal aggregate loss in case of Erlang-2 distributed claim sizes with mean 16 thousand PLZ, and give an explicit expression for the ruin probability ψE2(u).

In [64]:
R, t = sy.symbols("R theta")

eq = sy.Eq(16000 * R * (1 + t) + 1, 1 / ((1 - 8000 * R) ** 2))
# sy.solve(eq, R)[2].subs({t:0}).evalf()
eq1 = sy.solve(eq, R)[1]
eq2 = sy.solve(eq, R)[2]

# thetaList = [i / 1000 for i in range(1,10000)] # we assume that theta>0
# plt.plot(thetaList, [eq1.subs({t:theta})for theta in thetaList], color="b")
# plt.plot(thetaList, [eq2.subs({t:theta})for theta in thetaList], color="r")
# thetaList
eq1.subs({t:0.0001})

8.33259266253483e-9